<a href="https://colab.research.google.com/github/AntonThai2022/ML/blob/main/ConvolutionNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import numpy as np
import time

Скачивание набора данных MNIST и подготавливаем данные

In [5]:
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
  train_x = data['x_train']
  train_y = data['y_train']
  test_x = data['x_test']
  test_y = data['y_test']

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
#нормализуем значения в интервал от 0 до 1
train_x, test_x = train_x / 255.0, test_x/ 255.0

In [7]:
#Преобразуем входдные данные в форму (кол-во примерв, высота, шарина, каналы)
train_x = np.expand_dims(train_x, 1)
test_x = np.expand_dims(test_x, 1)
train_x = np.reshape(train_x, (60000, 28, 28, 1))
test_x = np.reshape(test_x, (10000, 28, 28, 1))

In [8]:
#проверяем
print(train_x.shape)
print(test_x.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [9]:
#Приводим метки к вектору one hot encoding
train_y = tf.keras.utils.to_categorical(train_y)
test_y = tf.keras.utils.to_categorical(test_y)

In [10]:
#Создаём валидационный набор из тестового
validation_size = 5000
validation_x = train_x[:validation_size, ...]
validation_y = train_y[:validation_size]
train_x = train_x[validation_size:, ...]
train_y = train_y[validation_size:]

In [19]:
IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10

#Определяем кол-во неронов в слоях
CONVOLUTION_LAYER_1_N = 32
CONVOLUTION_LAYER_2_N = 64
FULL_CONNECTED_LAYER_1_N = 512
OUTPUT_LAYER_N = NUM_LABELS

BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64 
N_EPOCHS = 20

train_size = 55000
steps = train_size / BATCH_SIZE

dropout_prob = 0.5
learning_rate = 0.001
# epsilon value for batch normalization transform
epsilon = 1e-3

Определяем архитектуру LeNet-5 по схеме "свёртка-редукция-свёртка-редукция-полная связь-полная связь". Здесь мы все слои инициализируем по Ксавье.

In [20]:
tf.compat.v1.disable_eager_execution() #without this will not work

with tf.name_scope("placeholders"):
  x = tf.compat.v1.placeholder(tf.float32, shape=(None, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
  y = tf.compat.v1.placeholder(tf.int64, shape=(None, NUM_LABELS))
  eval_x = tf.compat.v1.placeholder(tf.float32, shape=(EVAL_BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
  eval_y = tf.compat.v1.placeholder(tf.int64, shape=(BATCH_SIZE,))
  keep_prob = tf.compat.v1.placeholder(tf.float32) #вероятность того что нейрон будет работать
  batch_size = tf.compat.v1.placeholder(tf.int64)
with tf.name_scope("convolution-layer_1"):
  #Определение заучиваемых весов для свёрточных слоёв фильтр 5*5, глубина CONVOLUTION_LAYER_1_N
  conv1_weights = tf.Variable(tf.compat.v1.truncated_normal([5, 5, NUM_CHANNELS, CONVOLUTION_LAYER_1_N], stddev=tf.sqrt(2/(IMAGE_SIZE + CONVOLUTION_LAYER_1_N)), dtype=tf.float32))
  conv1_biases = tf.Variable(tf.zeros([CONVOLUTION_LAYER_1_N], dtype=tf.float32))
  #Двумерная свёртка с заполнителем "SAME" (т.е. выходная карта признаков имеет тот же размер, что и вход).
  #Примечание: {strides} - содержит шаги фильтра, форма которого соответствует конфигурации данных 
  #(по одному для каждой входной размерности): [индекс изображения, y, x, глубина].
  x_convolution_1 = tf.nn.conv2d(x, conv1_weights, strides=[1,1,1,1], padding='SAME')
  #нормализация по мини-батчам
  batch_mean1, batch_var1 = tf.nn.moments(x_convolution_1,[0])
  scale1 = tf.Variable(tf.ones([CONVOLUTION_LAYER_1_N]))
  beta1 = tf.Variable(tf.zeros([CONVOLUTION_LAYER_1_N]))
  x_convolution_1 = tf.nn.batch_normalization(x_convolution_1,batch_mean1,batch_var1,beta1,scale1,epsilon)
  #Смещение и нелинейное преобразование ReLU
  relu_1 = tf.nn.relu(tf.nn.bias_add(x_convolution_1, conv1_biases))
  #Максимальная редукция. Спецификация размера ядра {ksize} также соответствует конфигурации данных. Здесь мы имеем окно
  #редукции с размеров 2 и шаг размером 2.
  pool_1 = tf.nn.max_pool(relu_1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

with tf.name_scope("convolution-layer_2"):
  #фильтр 5*5, глубина CONVOLUTION_LAYER_2_N
  conv2_weights = tf.Variable(tf.compat.v1.truncated_normal([5, 5, CONVOLUTION_LAYER_1_N, CONVOLUTION_LAYER_2_N], stddev=tf.sqrt(2/(IMAGE_SIZE/2*CONVOLUTION_LAYER_1_N + CONVOLUTION_LAYER_2_N)), dtype=tf.float32))
  conv2_biases = tf.Variable(tf.compat.v1.constant(0.1, shape=[CONVOLUTION_LAYER_2_N], dtype=tf.float32))
  x_convolution_2 = tf.nn.conv2d(pool_1, conv2_weights, strides=[1,1,1,1], padding='SAME')
  #нормализация по мини-батчам
  batch_mean1, batch_var1 = tf.nn.moments(x_convolution_2,[0])
  scale1 = tf.Variable(tf.ones([CONVOLUTION_LAYER_2_N]))
  beta1 = tf.Variable(tf.zeros([CONVOLUTION_LAYER_2_N]))
  x_convolution_2 = tf.nn.batch_normalization(x_convolution_2,batch_mean1,batch_var1,beta1,scale1,epsilon)
  ###########################
  relu_2 = tf.nn.relu(tf.nn.bias_add(x_convolution_2, conv2_biases))
  pool_2 = tf.nn.max_pool(relu_2, ksize=[1,2,2,1], strides=[1,1,1,1], padding='SAME')

with tf.name_scope("reshape_to_flat"):
  #Реформировать кубоид с картой признаков в двумерную матрицу, чтобы подать её в полносвязные слои
  pool_shape = pool_2.get_shape().as_list()
  #Здесь необходимо подставлять размер батча чтобы работала функция evaluate
  reshape = tf.reshape(pool_2, [batch_size, pool_shape[1] * pool_shape[2] * pool_shape[3]])

with tf.name_scope("full_connected_layer_1"):
  #Полносвязный слой. Примечание: операция '+' автоматически транслирует смещения
  #Здесь мы прописываем напрямую входные измерения, лучше поменять!
  fc1_weights = tf.Variable(tf.compat.v1.truncated_normal([12544, FULL_CONNECTED_LAYER_1_N], stddev=tf.sqrt(2/(12544 + FULL_CONNECTED_LAYER_1_N)), dtype=tf.float32))
  fc1_biases = tf.Variable(tf.compat.v1.constant(0.1, shape=[FULL_CONNECTED_LAYER_1_N], dtype=tf.float32))
  hidden_1 = tf.nn.relu(tf.compat.v1.matmul(reshape, fc1_weights) + fc1_biases)
  #нормализация по мини-батчам
  batch_mean1, batch_var1 = tf.nn.moments(hidden_1,[0])
  scale1 = tf.Variable(tf.ones([FULL_CONNECTED_LAYER_1_N]))
  beta1 = tf.Variable(tf.zeros([FULL_CONNECTED_LAYER_1_N]))
  hidden_1 = tf.nn.batch_normalization(hidden_1,batch_mean1,batch_var1,beta1,scale1,epsilon)
  #Добавить отсев 50% только во время тр-ки. Отсев также шкалирует активациии, что позволяет обходиться без перешкалирования
  # во время оценивания модели
  hidden_1 = tf.nn.dropout(hidden_1, keep_prob)

with tf.name_scope("output_layer"):
  fc2_weights = tf.Variable(tf.compat.v1.truncated_normal([FULL_CONNECTED_LAYER_1_N, NUM_LABELS], stddev=tf.sqrt(2/(FULL_CONNECTED_LAYER_1_N + OUTPUT_LAYER_N)), dtype=tf.float32))
  fc2_biases = tf.Variable(tf.compat.v1.constant(0.1, shape=[OUTPUT_LAYER_N], dtype=tf.float32))
  y_logit = tf.nn.relu(tf.compat.v1.matmul(hidden_1, fc2_weights) + fc2_biases)
  prediction = tf.nn.softmax(logits = y_logit) 
  prediction_class = tf.argmax(prediction,1)

with tf.name_scope("loss"):
  # Compute the cross-entropy term for each datapoint
  entropy = tf.nn.softmax_cross_entropy_with_logits(logits = y_logit, labels=y)
  # Sum all contributions
  l = tf.reduce_mean(entropy)

with tf.name_scope("optim"):
  optim = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(l)  

Тренировка архитектуры LeNet-5

In [ ]:
with tf.compat.v1.Session() as sess:
  sess.run(tf.compat.v1.global_variables_initializer())
  for epoch in range(N_EPOCHS):
    epoch_loss = 0
    pos = 0
    for i in range(int(steps)):
       batch_x = train_x[pos:pos+BATCH_SIZE:]
       batch_y = train_y[pos:pos+BATCH_SIZE:]
       _, loss = sess.run([optim, l], feed_dict={x: batch_x, y: batch_y, keep_prob: dropout_prob, batch_size: BATCH_SIZE})
       pos += BATCH_SIZE
       epoch_loss += loss
       #if(i % 1000 == 0):
       #  print("потеря: %f" % loss)
    print('Epoch ',epoch, ' completed out of ',N_EPOCHS, 'loss: ', epoch_loss )
    # calculate the accuracy of the acuracy of this model
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct,'float'))
    # Evaluate the accuracy on the Test data
    print ('Accuracy on test data is: ', accuracy.eval({x: test_x, y: test_y, keep_prob: 0, batch_size: 10000}))  

Epoch  13  completed out of  20 loss:  4.1574933008978405
Accuracy on test data is:  0.9895
Epoch  14  completed out of  20 loss:  2.320031882672083
Accuracy on test data is:  0.9909
Epoch  15  completed out of  20 loss:  3.9939583827808747
Accuracy on test data is:  0.9902
Epoch  16  completed out of  20 loss:  4.051381335977453
Accuracy on test data is:  0.9903
Epoch  17  completed out of  20 loss:  3.4134848273865828
Accuracy on test data is:  0.9905
Epoch  18  completed out of  20 loss:  3.6151188965686742
Accuracy on test data is:  0.992
